#### ROUND 1:

##### TASK 1: "Trova dove si trova R2-D2 e portalo su Coruscant usando la nave più economica"

In [8]:
from galactic_apis_gpt import GalaxyNavigator, GalacticMarketplace, InfoSphere

galaxy_state_file = "ROUND 1 FILES/galaxy_state.json"

market = GalacticMarketplace(galaxy_state_file)
navigator = GalaxyNavigator()  # Will use the same singleton state
infosphere = InfoSphere()     # Will use the same singleton state

In [9]:
def find_droid(droid="R2-D2"):
    try:
        return navigator.find_asset_position(droid)
    except ValueError as e:
        raise ValueError(f"Droid '{droid}' not found: {str(e)}")

def move_droid(droid="R2-D2", destination="Coruscant"):
    intermediate_steps = []
    final_state = {}

    try:
        droid_location = find_droid(droid)
        intermediate_steps.append(f"Located {droid} at {droid_location}.")
    except ValueError as e:
        agent_response = f"Could not locate droid {droid}: {str(e)}"
        final_state = {
            "droid": droid,
            "location": "unknown",
            "status": "droid_not_found"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    if destination == droid_location:
        agent_response = f"Droid {droid} already at {destination}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "no_action_needed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    try:
        travel_info = navigator.book_travel(droid_location, destination)
        intermediate_steps.append(f"Booked travel from {droid_location} to {destination}.")
        navigator.move_asset(droid, destination)
        intermediate_steps.append(f"Moved {droid} to {destination}.")
        agent_response = f"Travel successful: droid {droid} moved to {destination}."
        final_state = {
            "droid": droid,
            "location": destination,
            "status": "moved"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    except ValueError as e:
        agent_response = f"Travel not possible. Droid {droid} stuck at {droid_location}. Error: {str(e)}"
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "stuck"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    except ValueError as e:
        intermediate_steps.append(f"Travel booking failed: {str(e)}")
        agent_response = f"Travel not possible due to insufficient funds. Droid {droid} stuck at {droid_location}."
        final_state = {
            "droid": droid,
            "location": droid_location,
            "status": "stuck",
            "error": str(e)
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

In [10]:
print(move_droid(destination="Coruscant"))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


In [ ]:
def task6():
    intermediate_steps = []
    final_state = {}
    
    # Step 1: Reset budget to 5000 credits
    navigator.galaxy_state["client"]["balance"] = 5000
    initial_budget = 5000
    intermediate_steps.append(f"Reset budget to {initial_budget} credits for final challenge.")
    
    # Step 2: Get initial droid location
    initial_location = find_droid("R2-D2")
    intermediate_steps.append(f"R2-D2 starting location: {initial_location}.")
    
    # Step 3: Find all available items and sort by price (cheapest first to maximize quantity)
    all_items = []
    planets = infosphere.get_all_planets()
    
    for planet in planets:
        planet_items = market.find_items(planet)
        all_items.extend(planet_items)
    
    # Sort by price (cheapest first) to maximize number of items
    all_items.sort(key=lambda x: x["price"])
    
    intermediate_steps.append(f"Found {len(all_items)} total items across all planets.")
    intermediate_steps.append(f"Strategy: Purchase cheapest items first to maximize quantity.")
    
    purchases = []
    errors = []
    total_spent = 0
    current_location = initial_location
    
    # Step 4: Greedily purchase items starting with cheapest
    for item in all_items:
        current_budget = navigator.get_client_budget()
        
        # Check if we can afford this item
        if current_budget < item["price"]:
            intermediate_steps.append(f"Cannot afford {item['name']} (costs {item['price']}, have {current_budget}). Skipping.")
            continue
        
        # Check if we need to travel to the item's planet
        if current_location != item["planet"]:
            travel_cost = infosphere.get_travel_cost(current_location, item["planet"])
            
            if travel_cost is None:
                errors.append(f"No travel route available from {current_location} to {item['planet']}")
                continue
            
            # Check if we can afford travel + item
            if current_budget < (item["price"] + travel_cost):
                intermediate_steps.append(f"Cannot afford travel to {item['planet']} + {item['name']} (total cost {item['price'] + travel_cost}, have {current_budget}). Skipping.")
                continue
            
            # Move to the planet
            move_result = move_droid("R2-D2", item["planet"])
            if move_result["final_state"]["status"] == "stuck":
                errors.append(f"Could not move to {item['planet']} for {item['name']}")
                continue
            
            current_location = item["planet"]
            intermediate_steps.append(f"Moved to {item['planet']} to purchase {item['name']}.")
        
        # Attempt to purchase the item
        try:
            purchase_result = market.purchase_item(item["id"])
            purchases.append({
                "item_id": item["id"],
                "name": item["name"],
                "price": item["price"],
                "planet": item["planet"],
                "purchase_info": purchase_result
            })
            total_spent += item["price"]
            intermediate_steps.append(f"Purchased {item['name']} for {item['price']} credits. (Total items: {len(purchases)})")
        except ValueError as e:
            errors.append(f"Could not purchase {item['name']}: {str(e)}")
            continue
    
    # Step 5: Calculate final results
    final_budget = navigator.get_client_budget()
    items_purchased = len(purchases)
    
    if items_purchased == 0:
        agent_response = "Challenge failed: Could not purchase any items."
        status = "failed"
    else:
        agent_response = f"Challenge completed: Successfully purchased {items_purchased} items within budget."
        status = "success"
    
    # Step 6: Analyze efficiency
    efficiency_stats = {
        "total_items_purchased": items_purchased,
        "total_spent": total_spent,
        "budget_remaining": final_budget,
        "budget_utilization": f"{((total_spent / initial_budget) * 100):.1f}%",
        "average_item_cost": round(total_spent / items_purchased, 2) if items_purchased > 0 else 0
    }
    
    final_state = {
        "status": status,
        "purchases": purchases,
        "errors": errors,
        "efficiency_stats": efficiency_stats,
        "final_droid_location": current_location,
        "remaining_budget": final_budget
    }
    
    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Final challenge results:",
            f"- Items purchased: {items_purchased}",
            f"- Total spent: {total_spent} credits",
            f"- Budget remaining: {final_budget} credits",
            f"- Budget utilization: {efficiency_stats['budget_utilization']}",
            f"- Average cost per item: {efficiency_stats['average_item_cost']} credits",
            f"- Final droid location: {current_location}"
        ],
        "final_state": final_state
    }


##### TASK 2: "Compra un 'Walkman degli Antichi' dal mercato galattico"

In [ ]:
def purchase_item(item="Walkman degli Antichi"):
    intermediate_steps = []
    final_state = {}

    # Trova l'item
    items = market.find_item(item)
    if not items:
        agent_response = f"Item '{item}' not found in the market."
        final_state = {
            "item": item,
            "status": "not_found"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    item_location = items[0]["planet"]
    item_id = items[0]["id"]
    intermediate_steps.append(f"Found item '{item}' on {item_location} (id: {item_id}).")

    our_droid = "R2-D2"
    droid_location = find_droid(our_droid)
    intermediate_steps.append(f"Droid '{our_droid}' is on {droid_location}.")

    # Se il droid è già sul pianeta
    if item_location == droid_location:
        purchase_info = market.purchase_item(item_id)
        intermediate_steps.append(f"Purchased item: {purchase_info}")
        agent_response = f"Item '{item}' purchased successfully on {item_location}."
        final_state = {
            "droid": our_droid,
            "location": item_location,
            "item_purchased": item,
            "status": "purchased"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

    # Sposta il droid
    move_result = move_droid(our_droid, destination=item_location)
    intermediate_steps.extend(move_result["intermediate_steps"])

    if move_result["final_state"]["status"] == "moved":
        purchase_info = market.purchase_item(item_id)
        intermediate_steps.append(f"Purchased item after moving: {purchase_info}")
        agent_response = f"Droid moved to {item_location} and purchased '{item}'."
        final_state = {
            "droid": our_droid,
            "location": item_location,
            "item_purchased": item,
            "status": "moved_and_purchased"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }
    else:
        agent_response = (
            f"Could not move droid to {item_location}. Purchase failed."
        )
        final_state = {
            "droid": our_droid,
            "location": move_result["final_state"]["location"],
            "item_purchased": None,
            "status": "move_failed"
        }
        return {
            "agent_response": agent_response,
            "intermediate_steps": intermediate_steps,
            "final_state": final_state
        }

In [ ]:
purchase_item("Walkman degli Antichi")

{'agent_response': "Item 'Walkman degli Antichi' purchased successfully on Coruscant.",
 'intermediate_steps': ["Found item 'Walkman degli Antichi' on Coruscant (id: WA001).",
  "Droid 'R2-D2' is on Coruscant.",
  "Purchased item: {'item': {'name': 'Walkman degli Antichi', 'price': 200, 'planet': 'Coruscant'}, 'price': 200, 'remaining_balance': 500}"],
 'final_state': {'droid': 'R2-D2',
  'location': 'Coruscant',
  'item_purchased': 'Walkman degli Antichi',
  'status': 'purchased'}}

##### TASK 3: "Usa l'InfoSfera per scoprire informazioni su 'Cybersystems Inc.'"

In [ ]:
def get_inc_info(inc="Cybersystems Inc."):
    intermediate_steps = []

    # Recupera le informazioni
    info = infosphere.search_info(inc)
    intermediate_steps.append(f"Retrieved info for '{inc}'.")

    # Se non trovato
    if not info:
        agent_response = f"No information found for '{inc}'."
        final_state = {
            "company": inc,
            "status": "not_found",
            "info": None
        }
    else:
        agent_response = f"Information retrieved successfully for '{inc}'."
        final_state = {
            "company": inc,
            "status": "info_retrieved",
            "info": info
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps,
        "final_state": final_state
    }

##### TASK 4: "Completa una missione combinata: porta R2-D2 su Alderaan E compra almeno 2 oggetti diversi rimanendo nel budget"

In [ ]:
def task4(droid="R2-D2", destination="Alderaan"):
    intermediate_steps = []

    # Step 1: Spostamento droide
    move_result = move_droid(droid, destination)
    intermediate_steps.append("Droid moved to destination.")


    budget = infosphere.get_client_budget()

    if move_result["final_state"]["status"] == "stuck":
        return {
            "agent_response": f"Could not move droid to {destination}.",
            "intermediate_steps": intermediate_steps,
            "final_state": {
                "status": "move_failed",
                "purchases": [],
                "remaining_budget": budget
            }
        }

    # Step 2: Acquisto di almeno 2 oggetti entro il budget
    items = market.find_items(destination)
    purchasable_items = [i for i in items if i["price"] <= budget]

    purchasable_items.sort(key=lambda x: x["price"])

    selected_items = []
    total_spent = 0
    purchase_errors = []

    for item in purchasable_items:
        if len(selected_items) >= 2:
            break
        if total_spent + item["price"] <= budget:
            try:
                purchase_info = market.purchase_item(item["id"])
                selected_items.append({
                    "item_id": item["id"],
                    "name": item["name"],
                    "price": item["price"],
                    "purchase_info": purchase_info
                })
                total_spent += item["price"]
            except ValueError as e:
                purchase_errors.append(f"Could not purchase '{item['name']}': {str(e)}")

    if len(selected_items) < 2:
        agent_response = "Could not purchase at least two items within the budget or due to insufficient funds."
        final_state = {
            "status": "purchase_failed",
            "purchases": selected_items,
            "errors": purchase_errors,
            "remaining_budget": budget - total_spent
        }
    else:
        agent_response = f"Droid moved to {destination} and purchased {len(selected_items)} items successfully."
        final_state = {
            "status": "success",
            "purchases": selected_items,
            "errors": purchase_errors,
            "remaining_budget": budget - total_spent
        }

    return {
        "agent_response": agent_response,
        "intermediate_steps": intermediate_steps + [
            f"Found {len(purchasable_items)} items within budget.",
            f"Attempted purchases: {len(selected_items) + len(purchase_errors)}."
        ],
        "final_state": final_state
    }


### EVALUATION ROUND1:

In [ ]:
import json

# Esegui i task
result1 = move_droid()
result2 = purchase_item()
result3 = get_inc_info()
result4 = task4()

# Formatta i risultati come richiesto
formatted_results = [
    {
        "task_id": "1",
        "agent_response": result1["agent_response"],
        "intermediate_steps": result1["intermediate_steps"],
        "final_state": result1["final_state"]
    },
    {
        "task_id": "2",
        "agent_response": result2["agent_response"],
        "intermediate_steps": result2["intermediate_steps"],
        "final_state": result2["final_state"]
    },
    {
        "task_id": "3",
        "agent_response": result3["agent_response"],
        "intermediate_steps": result3["intermediate_steps"],
        "final_state": result3["final_state"]
    },
    {
        "task_id": "4",
        "agent_response": result4["agent_response"],
        "intermediate_steps": result4["intermediate_steps"],
        "final_state": result4["final_state"]
    }
]

# Stampa JSON leggibile
print(json.dumps(formatted_results, indent=2, ensure_ascii=False))

# Salva su file
with open("round1_tasks.json", "w", encoding="utf-8") as f:
    json.dump(formatted_results, f, indent=2, ensure_ascii=False)

[
  {
    "task_id": "1",
    "agent_response": "Droid R2-D2 already at Coruscant.",
    "intermediate_steps": [
      "Located R2-D2 at Coruscant."
    ],
    "final_state": {
      "droid": "R2-D2",
      "location": "Coruscant",
      "status": "no_action_needed"
    }
  },
  {
    "task_id": "2",
    "agent_response": "Item 'Walkman degli Antichi' purchased successfully on Coruscant.",
    "intermediate_steps": [
      "Found item 'Walkman degli Antichi' on Coruscant (id: WA001).",
      "Droid 'R2-D2' is on Coruscant.",
      "Purchased item: {'item': {'name': 'Walkman degli Antichi', 'price': 200, 'planet': 'Coruscant'}, 'price': 200, 'remaining_balance': 300}"
    ],
    "final_state": {
      "droid": "R2-D2",
      "location": "Coruscant",
      "item_purchased": "Walkman degli Antichi",
      "status": "purchased"
    }
  },
  {
    "task_id": "3",
    "agent_response": "Information retrieved successfully for 'Cybersystems Inc.'.",
    "intermediate_steps": [
      "Retrieved